# Research Notes: Image Segmentation
---

This should be a continuation of the notes on image histograms.

Though notes on the topic may have ended prematurely, it is still advised to read the section first before reading this section as techniques may be referenced.

Here we define a function for creating dummy image histograms using the `StatsBase` module.

We also lift the `intervar(hist::FrequencyWeights, thresh::Int)` function from the previous section.

In [ ]:
using StatsBase

In [ ]:
# interclass variance function
function intervar(hist::FrequencyWeights, thresh::Int)
  ω_k = fweights(hist[1:thresh]).sum / hist.sum
  μ_T = mean(vec(1:length(hist)), hist) / hist.sum
  μ_0 = mean(vec(1:thresh), fweights(hist[1:thresh])) / hist.sum
  μ_1 = (μ_T-(ω_k*μ_0))/(1-ω_k)
    
  return (ω_k*(1-ω_k)) * ((μ_0 - μ_1)^2)
end

## Otsu's Method

Otsu's method for image segmentation has been used since its discovery in the late 1970s.

It splits the image into two groups, background and foreground, based on the frequencies of certain intensities of pixels which can be arranged into __image histograms__.

Otsu proposes, that the groups should be split in such a way that the variance _within_ the classes should be _minimized_.
Mathematically, since _total variance is constant_, minimizing the __intraclass variance__ means _maximizing_ the __interclass variance__ or variance _between_ classes.

Below shows an implementation in Julia of the algorithm referred to.

In [ ]:
# Plain implementation of Otsu's method
function otsu_segment(hist::FrequencyWeights)
    # set variance to the lowest reasonable variance
    max_var = 0
    
    # optimal thresh bin
    thresh = 0
    
    # iterate through entire histogram
    for curr in 1:length(hist)
        curr_var = intervar(hist, curr)
        
        # if the curr variance > max (recorded) variance
        # update max_var
        if max_var < curr_var
            max_var = curr_var
            thresh = curr
        end
    end
    
    return thresh
end

In [ ]:
# Plain modification to the above implementation
# Assumes that variance is strictly increasing to optimum thresh
# and strictly decreases afterwards

# breaks at "maximum"
function otsu_segment_break(hist::FrequencyWeights)
    # same local variable defns as above
    max_var = 0; thresh = 0
    
    # iterate over histogram
    for curr in 1:length(hist)
        curr_var = intervar(hist, curr)
        
        # if the curr variance > max (recorded) variance
        if max_var < curr_var
            max_var = curr_var
            thresh = curr
            
        # if the curr_var less than or equal to the recorded maximum
        # then break out of loop
        else
            break
        end
    end
    
    return thresh
end

## Otsu-Checkpoints by Alsaeed (2016)

This method uses the concept of trisecting a search array and looking for regions of maximum variance.
It is assumed that the algorithm was formed from the fact that interclass variance is found to be __concave__ around the optimal threshold.

Since the method searches for regions with maximal variance and then __truncates__ the search array, it can be performed __recursively__.

Below, we build a _simple_ function for determining the direction of maximal variance.
This only uses the fact of __concavity__ around the threshold.

In [ ]:
# simple directions to optimal threshold

# returns `dir::Int` with respect to `thresh::Int`
# if `dir == 1` then optimal thresh is higher
# elseif `dir == -1` then dir is lower
# otherwise, `thresh` is the optimal threshold
function var_dir(hist::FrequencyWeights, thresh::Int)
    if intervar(hist, thresh+1) > intervar(hist, thresh)
        return 1
    elseif intervar(hist, thresh-1) > intervar(hist, thresh)
        return -1
    else
        return 0
    end
end

Next, we define the processes for the proposed "Otsu-checkopoints" algorithm (AlSaeed et al., 2016) in the next code cell.

I consider the algorithm to be a _range-reduction_ algorithm since it reduces the test range into smaller sub-ranges.
There are two parts to the said reduction, however:

1. First, is "peak area" definition
2. Even trisection

The first part splits the image histogram into four sections, roughly, formed by the division of the peaks and the mean intensity.

In [ ]:
# proposed otsu-checkpoints algorithm

function alsaeed_otsu(hist::FrequencyWeights)
    # ready the bins
    bins = vec(1:length(hist)) ; init_max_c = 0;
    
    # get the endpoints of the four histogram segments
    μ_T = Int(floor(mean(bins, hist)))
    μ_0 = Int(floor(mean(bins[1:μ_T], fweights(hist[1:μ_T]))))
    μ_1 = Int(floor(mean(bins[μ_T:end], fweights(hist[μ_T:end]))))
    
    # the constructed regions are [1, μ_0] , [μ_0, μ_T], [μ_T, μ_1], and [μ_1, end]
    # determine which of the checkpoint thresholds splits the histogram
    # with the greatest interclass variance
    
    # still assume interclass concavity around the optimal threshold
    # for faster results
    if intervar(hist, μ_0) > intervar(hist, μ_T)
        init_max_c = μ_0
    elseif intervar(hist, μ_T) > intervar(hist, μ_1)
        init_max_c = μ_T
    else
        init_max_c = μ_1
    end
    
    # locally define a recursive function for the second type of range reduction used
    function alsaeed_reduction(r_hist::FrequencyWeights)
        # define measures of position
        l = 1; r = length(r_hist); range = r-l
        
        # if the length of the reduced histogram is 1, return the element
        
        # define middle checkpoints in range
        ch2 = l + range >> 2
        ch3 = l + range >> 1
        ch4 = r - range >> 2
        
        # determine which of the three splits the histogram with greatest interclass variance
        if intervar(r_hist, ch2) > intervar(r_hist, ch3)
            loc_dir = var_dir(r_hist, ch2)
            if loc_dir < 0
                return alsaeed_reduction(fweights(r_hist[l:ch2]))
            elseif loc_dir > 0
                return alseed_reduction(fweights(r_hist[ch2:ch3]))
        elseif intervar(r_hist, ch3) > intervar(r_hist, ch4)
            return # range with ch3
        else
            return # range with ch4
        end
        
    end # local reduction function end
end # function end

## Proposed Modification to Otsu (Binary search)

This method is slightly different to the method of Alseed above.
The only difference is that this proposed method will perform a "binary search" instead of trisecting the search array.
The implementation will be compared against the two algorithm above.